In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import csv
import os
from datetime import datetime
import matplotlib.pyplot as plt

CSV_FILE = 'bmi_data.csv'

class BMICalculatorApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("🎯 BMI Calculator")
        self.geometry("480x500")
        self.configure(bg="#f0f8ff")  # Light blue background
        self.data = self.load_data()
        
        self.selected_user = tk.StringVar()

        title_label = tk.Label(self, text="💪 BMI Calculator", font=("Arial", 18, "bold"), bg="#f0f8ff", fg="#2c3e50")
        title_label.pack(pady=10)

        tk.Label(self, text="Enter or Select Username:", bg="#f0f8ff", fg="#2c3e50", font=("Arial", 12)).pack()
        self.user_combo = ttk.Combobox(self, textvariable=self.selected_user, state="normal", width=30)
        self.user_combo['values'] = sorted(self.data.keys())
        self.user_combo.set('')
        self.user_combo.pack(pady=5)

        self.add_labeled_entry("Weight (kg):", "weight_entry")
        self.add_labeled_entry("Height (cm):", "height_entry")

        self.calc_btn = tk.Button(self, text="✅ Calculate BMI", bg="#27ae60", fg="white",
                                  font=("Arial", 11, "bold"), command=self.calculate_bmi)
        self.calc_btn.pack(pady=10)

        self.result_label = tk.Label(self, text="", font=("Arial", 14), bg="#f0f8ff", fg="#2c3e50")
        self.result_label.pack(pady=5)

        self.history_btn = tk.Button(self, text="📈 Show BMI History & Trend", bg="#2980b9", fg="white",
                                     font=("Arial", 11, "bold"), command=self.show_history)
        self.history_btn.pack(pady=10)

    def add_labeled_entry(self, label_text, attr_name):
        label = tk.Label(self, text=label_text, bg="#f0f8ff", fg="#2c3e50", font=("Arial", 12))
        label.pack()
        entry = tk.Entry(self)
        entry.pack()
        setattr(self, attr_name, entry)

    def load_data(self):
        data = {}
        if os.path.exists(CSV_FILE):
            try:
                with open(CSV_FILE, newline='') as f:
                    reader = csv.DictReader(f)
                    for row in reader:
                        user = row['username']
                        if user not in data:
                            data[user] = []
                        data[user].append({
                            'date': row['date'],
                            'bmi': float(row['bmi']),
                            'weight': float(row['weight']),
                            'height': float(row['height'])
                        })
            except Exception as e:
                print(f"Warning: Failed to load data: {e}")
        return data

    def save_data(self):
        rows = []
        for user, records in self.data.items():
            for rec in records:
                rows.append({
                    'username': user,
                    'date': rec['date'],
                    'bmi': rec['bmi'],
                    'weight': rec['weight'],
                    'height': rec['height']
                })
        try:
            with open(CSV_FILE, 'w', newline='') as f:
                fieldnames = ['username', 'date', 'bmi', 'weight', 'height']
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
        except Exception as e:
            messagebox.showerror("Save Error", f"Failed to save data:\n{e}")

    def calculate_bmi(self):
        user = self.selected_user.get().strip()
        weight_str = self.weight_entry.get().strip()
        height_str = self.height_entry.get().strip()

        if not user:
            messagebox.showerror("Input Error", "Please enter or select a username.")
            return

        try:
            weight = float(weight_str)
            height_cm = float(height_str)
        except ValueError:
            messagebox.showerror("Input Error", "Weight and Height must be valid numbers.")
            return

        if not (10 <= weight <= 500):
            messagebox.showerror("Input Error", "Weight should be between 10 and 500 kg.")
            return
        if not (50 <= height_cm <= 300):
            messagebox.showerror("Input Error", "Height should be between 50 and 300 cm.")
            return

        height_m = height_cm / 100
        bmi = weight / (height_m ** 2)
        bmi_rounded = round(bmi, 2)
        category = self.get_bmi_category(bmi)

        self.result_label.config(text=f"BMI: {bmi_rounded} ({category})")

        date_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        if user not in self.data:
            self.data[user] = []
        self.data[user].append({
            'date': date_str,
            'bmi': bmi_rounded,
            'weight': weight,
            'height': height_cm
        })

        self.save_data()

        if user not in self.user_combo['values']:
            new_values = list(self.user_combo['values']) + [user]
            self.user_combo['values'] = sorted(new_values)

    def get_bmi_category(self, bmi):
        if bmi < 18.5:
            return "Underweight"
        elif bmi < 25:
            return "Normal weight"
        elif bmi < 30:
            return "Overweight"
        else:
            return "Obesity"

    def show_history(self):
        user = self.selected_user.get().strip()
        if not user:
            messagebox.showerror("Input Error", "Please enter or select a username to see history.")
            return
        if user not in self.data or len(self.data[user]) == 0:
            messagebox.showinfo("No Data", f"No BMI history found for user '{user}'.")
            return

        records = sorted(self.data[user], key=lambda x: x['date'])
        dates = [rec['date'] for rec in records]
        bmis = [rec['bmi'] for rec in records]

        plt.figure(figsize=(8, 4))
        plt.plot(dates, bmis, marker='o', linestyle='-', color='green')
        plt.xticks(rotation=45, ha='right')
        plt.title(f'📊 BMI Trend for {user}', fontsize=14)
        plt.xlabel('Date', fontsize=12)
        plt.ylabel('BMI', fontsize=12)
        plt.grid(True)
        plt.tight_layout()
        plt.show()


# Run the app in Jupyter
if __name__ == '__main__':
    app = BMICalculatorApp()
    app.mainloop()

